In [1]:
import nest_asyncio
nest_asyncio.apply()
import pandas as pd
from scapy.all import *
import glob
import numpy as np
import os

In [2]:
def Average(lst): 
    if lst:
        return sum(lst) / len(lst)
    else:
        return 0

In [39]:
readFolder = '4GQoS'
saveFolder='4GQoS12'

folder = 'fol_1_mode_5g_trace_4g_Case_2_host_1_algo_bba_protocol_tcp_server_WSGI'
os.mkdir(saveFolder+'/'+str(folder)+'/')
location = readFolder+'/'+folder+''
pcap =rdpcap(location+'/h1_ap.pcap')

In [4]:
def EMA(mArray):
    a = 0.99
    emaArray = []
    emaArray.append(mArray[0])
    j=1
    for i in range(1, len(mArray)):
        emaArray.append(mArray[i] * (1 - a) + emaArray[j - 1] * a) 
        j+=1
    return sum(emaArray)

In [6]:
def DEMA(mArray):
    a = 0.99
    emaArray = []
    emaArray.append(mArray[0])
    j=1
    for i in range(1, len(mArray)):
        emaArray.append( (a * mArray[i]) + ((1-a) + emaArray[j -1]) ) 
        j+=1
    return sum(emaArray)

In [40]:
for i in range(1, 2, +1):# Here I am taking just two seconds 1, 2, jump of 1
    serverIP = '10.0.0.1'
    connection=[];       
    flag=True
    for ppp in pcap:                               
        if (ppp.haslayer(TCP)):
            if (ppp[IP].dst == '10.0.0.1'):
                if (ppp[TCP].flags == 'FA' or ppp[TCP].flags == 'F'):
                    connection.append(ppp[TCP].sport)

    start=int(0)
    end_time=140
    begin=0
    windows_size=i
    df= pd.DataFrame(columns=['Time',
                          'interIntervalTimeDP',
                          'interIntervalTimeDP_GT100',
                          'AvgTimeBWDPK',
                          'AvgTimeBWDPK_GT100',
                          'DThroughput',
                          'DTTime',
                          'DTotal_Packets',
                          'DTotal_Packets_GT100',
                          'STD_Less',
                          'STD_GT',
                          'EMA',
                          'DEMA',
                          'CUSUM',
                          '10P',
                              '20P',
                              '30P',
                              '40P',
                              '50P',
                              '60P',
                              '70P',
                              '80P',
                              '90P',
                              '10T',
                              '20T',
                              '30T',
                              '40T',
                              '50T',
                              '60T',
                              '70T',
                              '80T',
                              '90T'
                         
                         ])
    for i in np.arange(start, end_time, windows_size):
        dl_05=0; dc_05=0;dc_gt100_05=0;da_05=0;
        dt_05=begin;
        packets_time_05=[];d_ia_05=[];d_pk_size_05=[];d_pk_size_gt100_05=[];
        ul_05=0;uc_gt100_05=0;ua_05=0;
        ut_05=begin;uc_05=0
        u_ia_05=[];u_pk_size_05=[];u_pk_size_gt100_05=[];d_packets_gt_100_time=[];
        u_packets_time_05=[];u_packets_gt_100_time=[]
        packet_size = []
        tp_distribution = []
        tm_difference = []
        for p in pcap:
            
            if (p.haslayer(TCP)) and ((float(p.time-pcap[0].time))>= i) and  ((float(p.time-pcap[0].time))< i+windows_size) and (p[TCP].sport in connection or p[TCP].dport in connection):
                
                if(p[IP].src== '10.0.0.1'):# downlink
                    dl_05=dl_05+(int(len(p[IP]))*8)                   
                    dc_05=dc_05+1
                    packet_size.append(int(len(p[IP]))) # Appended to get percentile distibutions
                    if int(len(p))>=100:# ignore ack pk
                        dc_gt100_05=dc_gt100_05+1
                        
                    packets_time_05.append(float(p.time))                    
                    da_05=float(p.time-dt_05)
                    d_ia_05.append(da_05)
                    dt_05=p.time                    
                    d_pk_size_05.append(int(len(p[IP])))
                    #print(p.show())
                    if int(len(p))>=100:# ignore ack pk
                        d_packets_gt_100_time.append(p.time)
                        d_pk_size_gt100_05.append(int(len(p[IP])))


        #-----------------------------------Download Block------------------------     
        if packets_time_05:
            
            if dc_05>2:
                tpTime_05 = packets_time_05[-1]- packets_time_05[0]
                std_less=np.std(d_pk_size_05)
            else:
                std_less=0

            if dc_gt100_05>2:
                #print(dc_gt100_05)
                tpTime_05_GT100 = d_packets_gt_100_time[-1]- d_packets_gt_100_time[0]
                AvgTime_List_GT100 = [d_packets_gt_100_time[i + 1] - d_packets_gt_100_time[i] for i in range(len(d_packets_gt_100_time)-1)]
                AvgTime_GT100 = Average(AvgTime_List_GT100)
                std_grt=np.std(d_pk_size_gt100_05)
            else:
                AvgTime_GT100 =0
                tpTime_05_GT100=0
                std_grt=0


            TP_05=dl_05/tpTime_05
            TP_05_Time=dl_05/windows_size
            #print('TP: '+str(TP_05_Time)) D
            time_between_each_packets_05 = [packets_time_05[i + 1] - packets_time_05[i] for i in range(len(packets_time_05)-1)]
            
            #Below are shifts functions
            
            ema = EMA(time_between_each_packets_05)
            dema = DEMA(time_between_each_packets_05)
            cusum = sum(np.cumsum(time_between_each_packets_05))
            #np_psize = np.array(packet_size)
            #Packet Size Distributions
            pz_10 = np.percentile(packet_size,10)
            pz_20 = np.percentile(packet_size,20)
            pz_30 = np.percentile(packet_size,30)
            pz_40 = np.percentile(packet_size,40)
            pz_50 = np.percentile(packet_size,50)
            pz_60 = np.percentile(packet_size,60)
            pz_70 = np.percentile(packet_size,70)
            pz_80 = np.percentile(packet_size,80)
            pz_90 = np.percentile(packet_size,90)
            
            # Throughput Distribution ---- Block -----
           
            packet_size.pop(-1) # Poped becase time and size not equal
            smList = []            
            sm =0            
            for tp in range (0, len(packet_size)):
                sm+=packet_size[tp]*8               
                smList.append(sm)
            #print(smList)
            
            # Why this loop, bcz list out of range
            p=0
            dif = []
            ttl = len(packets_time_05)
            for pt in range(0, ttl-1):
                pre =packets_time_05[p]
                nxt = packets_time_05[pt+1]
                dif.append(nxt-pre)
            
            # For dividing times
            for d in range(0,len(smList)):
                tp_distribution.append(smList[d]/dif[d])
                
            
           
            
            tp_10 = np.percentile(tp_distribution,10)
            tp_20 = np.percentile(tp_distribution,20)
            tp_30 = np.percentile(tp_distribution,30)
            tp_40 = np.percentile(tp_distribution,40)
            tp_50 = np.percentile(tp_distribution,50)
            tp_60 = np.percentile(tp_distribution,60)
            tp_70 = np.percentile(tp_distribution,70)
            tp_80 = np.percentile(tp_distribution,80)
            tp_90 = np.percentile(tp_distribution,90)
        
            #print(tp_40)
        
            # Throughput Distribution ---- Block Ends ----
            
            
            average_05 = Average(time_between_each_packets_05)

            tms_string=''
            for tms in packets_time_05:
                tms_string = tms_string + str(tms)+'~'

            pkt_string=''
            for pk in d_pk_size_05:
                pkt_string = pkt_string + str(pk)+'~'

            d_packet_gt100_time_str=''
            for dpg100t in d_packets_gt_100_time:
                d_packet_gt100_time_str=d_packet_gt100_time_str+str(dpg100t)+'~'

            d_packet_gt100_str=''
            for dpg100 in d_pk_size_gt100_05:
                d_packet_gt100_str=d_packet_gt100_str+str(dpg100)+'~'
                
            df = df.append(pd.Series([i,
                                      tpTime_05,
                                      tpTime_05_GT100,
                                      average_05,
                                      AvgTime_GT100,
                                      TP_05,
                                      TP_05_Time,
                                      dc_05,
                                      dc_gt100_05,
                                      std_less,
                                      std_grt,
                                      ema,
                                      dema,
                                      cusum,
                                      pz_10,
                                      pz_20,
                                      pz_30,
                                      pz_40,
                                      pz_50,
                                      pz_60,
                                      pz_70,
                                      pz_80,
                                      pz_90,
                                      tp_10,
                                      tp_20,
                                      tp_30,
                                      tp_40,
                                      tp_50,
                                      tp_60,
                                      tp_70,
                                      tp_80,
                                      tp_90

                                     ], index=df.columns ), ignore_index=True)
            df.to_csv (saveFolder+'/'+folder+'/12qos_'+str(windows_size)+'.csv', index = None, header=True)
       